In [1]:
import requests as rq
import pandas as pd
import datetime as dt
import os

## Descarga de datos

Variables

In [14]:
url = 'https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_aperturas.xls'
fecha = dt.datetime.today().strftime('%Y-%m-%d') 


In [4]:
# Descarga de datos mediante el requests
try:
    r = rq.get(url)
    r.raise_for_status()
except rq.exceptions.HTTPError as err:
    print(err)



# Guardado de datos en el directorio /Data_cruda y se renombra el archivo agregando la fecha de descarga
if len(os.listdir('Data_cruda')) == 0:
    open(f'Data/sh_ipc_aperturas_{fecha}.xls', 'wb').write(r.content)
    print('El archivo se ha descargado correctamente')
else:
    for file in os.scandir('Data_cruda'):
        if file.name == f'sh_ipc_aperturas_{fecha}.xls':
            print('El archivo ya existe')
            break
        else:
            open(f'Data_cruda/sh_ipc_aperturas_{fecha}.xls', 'wb').write(r.content)
            print('El archivo se ha descargado correctamente')      
   



El archivo se ha descargado correctamente
El archivo ya existe


## Tratamiento de datos

Variables

In [5]:
file = f'Data_cruda/sh_ipc_aperturas_{fecha}.xls'

Creacion de datasets para su posterior analisis

In [6]:
df = pd.read_excel(file, sheet_name='Índices aperturas', skiprows=4, parse_dates=True)


In [7]:
df.shape

(297, 72)

In [8]:
df.info() #Tipo de datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 72 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   269 non-null    object
 1   Unnamed: 1   266 non-null    object
 2   Unnamed: 2   266 non-null    object
 3   Unnamed: 3   266 non-null    object
 4   Unnamed: 4   266 non-null    object
 5   Unnamed: 5   266 non-null    object
 6   Unnamed: 6   266 non-null    object
 7   Unnamed: 7   266 non-null    object
 8   Unnamed: 8   266 non-null    object
 9   Unnamed: 9   266 non-null    object
 10  Unnamed: 10  266 non-null    object
 11  Unnamed: 11  266 non-null    object
 12  Unnamed: 12  266 non-null    object
 13  Unnamed: 13  266 non-null    object
 14  Unnamed: 14  266 non-null    object
 15  Unnamed: 15  266 non-null    object
 16  Unnamed: 16  266 non-null    object
 17  Unnamed: 17  266 non-null    object
 18  Unnamed: 18  266 non-null    object
 19  Unnamed: 19  266 non-null    

In [9]:
df.isna().sum().sum() #Cantidad de valores nulos

2229

- La pestaña de datos 'indices aperturas' contiene varias row datos NaN, por lo que se eliminan para facilitar el analisis.
- Las columnas de datos estan en string, por lo que se convertiran a float para poder realizar operaciones matematicas.
- Se nombran las columnas de la pestaña 'indices aperturas' para facilitar el analisis.
- Debido a la complejidad del dataset y su estructura se decide dividir las regiones en dataset individuales.
- Realizamos un transpose de los datasets para facilitar el analisis. Quedando como nombre de los campos el nombre del indice y como indice la fecha.
- Se genera un unico CSV con los datos de los indices de apertura de cada region con un campo indicacion region, esto facilitara luego la busqueda de informacion en la DDBB.


In [10]:
df = df.dropna()
df.columns = ['Columna_'+str(i) for i in range(1, len(df.columns)+1)]
df = df.reset_index(drop=True)
df.head()

,Columna_1,Columna_2,Columna_3,Columna_4,Columna_5,Columna_6,Columna_7,Columna_8,Columna_9,Columna_10,...,Columna_63,Columna_64,Columna_65,Columna_66,Columna_67,Columna_68,Columna_69,Columna_70,Columna_71,Columna_72
0,Región GBA,2016-12-01 00:00:00,2017-01-01 00:00:00,2017-02-01 00:00:00,2017-03-01 00:00:00,2017-04-01 00:00:00,2017-05-01 00:00:00,2017-06-01 00:00:00,2017-07-01 00:00:00,2017-08-01 00:00:00,...,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00,2022-05-01 00:00:00,2022-06-01 00:00:00,2022-07-01 00:00:00,2022-08-01 00:00:00,2022-09-01 00:00:00,2022-10-01 00:00:00
1,Nivel general,100,101.313,103.8085,106.2627,109.0613,110.4607,111.9943,113.9199,115.6031,...,593.4219,620.5007,661.8688,702.829,736.3637,776.6701,833.8894,891.851,945.6836,1007.6275
2,Alimentos y bebidas no alcohólicas,100,101.3069,103.1646,106.7878,109.2144,110.3615,111.7923,113.1579,115.6549,...,627.7655,681.4654,722.8456,765.7118,799.1293,838.7821,888.2122,949.1534,1016.1594,1084.2719
3,Alimentos,100,101.2848,103.3191,107.1358,109.6145,110.6764,112.1594,113.3581,115.9763,...,645.0681,703.6564,747.2396,792.709,826.4546,866.4017,917.1386,980.6468,1051.1431,1119.2361
4,Pan y cereales,100,101.5263,103.7498,106.6043,108.8026,109.7644,111.8159,113.7735,115.1724,...,576.2742,609.602,680.0321,736.4364,767.4221,805.117,861.7042,921.9103,974.7834,1034.9946


Obtenemos las regiones y sus index

In [11]:
regiones = {}
for row in df.Columna_1:
    if 'Región' in row:
        regiones[row] = df[df.Columna_1 == row].index[0]

print(regiones)
        


{'Región GBA': 0, 'Región Pampeana': 46, 'Región Noroeste': 90, 'Región Noreste': 134, 'Región Cuyo': 178, 'Región Patagonia': 222}


### Generamos los dataset por region

Cada dataset contiene los datos de una region, con sus respectivos indices de apertura. Se guardan en formato csv en /Dataframe.

In [16]:
region_gba = df[regiones["Región GBA"] : regiones["Región Pampeana"]]
region_gba = region_gba.transpose()
region_gba.columns = region_gba.iloc[0]
region_gba = region_gba.drop(region_gba.index[0])
region_gba = region_gba.reset_index(drop=True)
region_gba.rename(columns={"Región GBA": "fecha"}, inplace=True)
region_gba.columns = [c.replace(" ", "_") for c in list(region_gba.columns)]
region_gba.columns = [c.replace(",", "") for c in list(region_gba.columns)]
region_gba.columns = region_gba.columns.str.lower()
region_gba["fecha"] = pd.to_datetime(region_gba["fecha"], format="%Y-%m-%d")
region_gba["region"] = "Region GBA"

region_gba.to_csv(rf"./Dataframe/region_gba_{fecha}.csv", index=False)

In [20]:
region_gba.head()

,fecha,nivel_general,alimentos_y_bebidas_no_alcohólicas,alimentos,pan_y_cereales,carnes_y_derivados,leche_productos_lácteos_y_huevos,aceites_grasas_y_manteca,frutas,verduras_tubérculos_y_legumbres,...,recreación_y_cultura,equipos_audiovisuales_fotográficos_y_de_procesamiento_de_la_información,servicios_recreativos_y_culturales,periódicos_diarios_revistas_libros_y_artículos_de_papelería,educación,restaurantes_y_hoteles,restaurantes_y_comidas_fuera_del_hogar,bienes_y_servicios_varios,cuidado_personal,region
0,2016-12-01,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,Region GBA
1,2017-01-01,101.313,101.3069,101.2848,101.5263,98.9006,102.1541,101.683,103.5925,104.6613,...,103.243,101.4745,103.9842,102.3552,100.4869,103.2868,102.273,102.117,101.4831,Region GBA
2,2017-02-01,103.8085,103.1646,103.3191,103.7498,98.8632,105.2875,102.9812,104.5837,111.8879,...,103.3096,98.7331,105.3351,104.8476,104.936,105.1373,104.2501,104.1416,103.3753,Region GBA
3,2017-03-01,106.2627,106.7878,107.1358,106.6043,104.1811,108.0146,104.9851,102.2939,120.5272,...,104.8017,99.9789,108.1269,105.6987,111.9338,106.1813,106.2338,105.5869,104.9077,Region GBA
4,2017-04-01,109.0613,109.2144,109.6145,108.8026,106.8365,113.7921,107.6769,99.354,121.4805,...,108.3531,100.3028,115.1047,106.5042,116.2786,108.3494,108.6171,107.5817,107.0113,Region GBA


In [19]:
region_pampeana = df[regiones["Región Pampeana"] : regiones["Región Noroeste"]]
region_pampeana = region_pampeana.transpose()
region_pampeana.columns = region_pampeana.iloc[0]
region_pampeana = region_pampeana.drop(region_pampeana.index[0])
region_pampeana = region_pampeana.reset_index(drop=True)
region_pampeana.rename(columns={"Región Pampeana": "fecha"}, inplace=True)
region_pampeana.columns = [
    c.replace(" ", "_") for c in list(region_pampeana.columns)
]
region_pampeana.columns = [
    c.replace(",", "") for c in list(region_pampeana.columns)
]
region_pampeana.columns = region_pampeana.columns.str.lower()
region_pampeana["fecha"] = pd.to_datetime(
    region_pampeana["fecha"], format="%Y-%m-%d"
)
region_pampeana["region"] = "Region Pampeana"

region_pampeana.to_csv(rf"./Dataframe/region_pampeana_{fecha}.csv", index=False)

region_noroeste = df[regiones["Región Noroeste"] : regiones["Región Noreste"]]
region_noroeste = region_noroeste.transpose()
region_noroeste.columns = region_noroeste.iloc[0]
region_noroeste = region_noroeste.drop(region_noroeste.index[0])
region_noroeste = region_noroeste.reset_index(drop=True)
region_noroeste.rename(columns={"Región Noroeste": "fecha"}, inplace=True)
region_noroeste.columns = [
    c.replace(" ", "_") for c in list(region_noroeste.columns)
]
region_noroeste.columns = [
    c.replace(",", "") for c in list(region_noroeste.columns)
]
region_noroeste.columns = region_noroeste.columns.str.lower()
region_noroeste["fecha"] = pd.to_datetime(
    region_noroeste["fecha"], format="%Y-%m-%d"
)
region_noroeste["region"] = "Region Noroeste"

region_noroeste.to_csv(rf"./Dataframe/region_noroeste_{fecha}.csv", index=False)

region_noreste = df[regiones["Región Noreste"] : regiones["Región Cuyo"]]
region_noreste = region_noreste.transpose()
region_noreste.columns = region_noreste.iloc[0]
region_noreste = region_noreste.drop(region_noreste.index[0])
region_noreste = region_noreste.reset_index(drop=True)
region_noreste.rename(columns={"Región Noreste": "fecha"}, inplace=True)
region_noreste.columns = [c.replace(" ", "_") for c in list(region_noreste.columns)]
region_noreste.columns = [c.replace(",", "") for c in list(region_noreste.columns)]
region_noreste.columns = region_noreste.columns.str.lower()
region_noreste["fecha"] = pd.to_datetime(region_noreste["fecha"], format="%Y-%m-%d")
region_noreste["region"] = "Region Noreste"

region_noreste.to_csv(rf"./Dataframe/region_noreste_{fecha}.csv", index=False)

region_cuyo = df[regiones["Región Cuyo"] : regiones["Región Patagonia"]]
region_cuyo = region_cuyo.transpose()
region_cuyo.columns = region_cuyo.iloc[0]
region_cuyo = region_cuyo.drop(region_cuyo.index[0])
region_cuyo = region_cuyo.reset_index(drop=True)
region_cuyo.rename(columns={"Región Cuyo": "fecha"}, inplace=True)
region_cuyo.columns = [c.replace(" ", "_") for c in list(region_cuyo.columns)]
region_cuyo.columns = [c.replace(",", "") for c in list(region_cuyo.columns)]
region_cuyo.columns = region_cuyo.columns.str.lower()
region_cuyo["fecha"] = pd.to_datetime(region_cuyo["fecha"], format="%Y-%m-%d")
region_cuyo["region"] = "Region Cuyo"

region_cuyo.to_csv(rf"./Dataframe/region_cuyo_{fecha}.csv", index=False)

region_patagonica = df[regiones["Región Patagonia"] :]
region_patagonica = region_patagonica.transpose()
region_patagonica.columns = region_patagonica.iloc[0]
region_patagonica = region_patagonica.drop(region_patagonica.index[0])
region_patagonica = region_patagonica.reset_index(drop=True)
region_patagonica.rename(columns={"Región Patagonia": "fecha"}, inplace=True)
region_patagonica.columns = [
    c.replace(" ", "_") for c in list(region_patagonica.columns)
]
region_patagonica.columns = [
    c.replace(",", "") for c in list(region_patagonica.columns)
]
region_patagonica.columns = region_patagonica.columns.str.lower()
region_patagonica["fecha"] = pd.to_datetime(
    region_patagonica["fecha"], format="%Y-%m-%d"
)
region_patagonica["region"] = "Region Patagonia"

region_patagonica.to_csv(
    rf"./Dataframe/region_patagonica_{fecha}.csv", index=False
)